In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import optimize

from sklearn.metrics import accuracy_score, f1_score
from pymoo.core.problem import Problem
from evoman.environment import Environment
from fitness_functions import original_fitness, individual_gain
from utils import simulation, verify_solution, init_env, run_pymoo_algorithm, initialise_script
import math


pygame 2.5.1 (SDL 2.28.2, Python 3.9.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
solutions = []
for file in os.listdir('solutions_beats_5_enemies'):
    if file.startswith('pymoo'):
        new_solution = []
        with open(f'solutions_beats_5_enemies/{file}') as f:
            solutions.append(f.read().splitlines())
            
solutions = np.array(solutions, dtype=float)

In [3]:
seed = 42
POP_SIZE = 1
ENEMIES = [1, 2, 3, 4, 5, 6, 7, 8]
n_hidden_neurons = 10


In [4]:
class objectives(Problem):
    enemies: list[int]
    env: Environment

    def __init__(self, env: Environment, n_genes: int, enemies: list[int], n_objectives):
        self.env = env
        self.enemies = enemies
        super().__init__(n_var=n_genes, n_obj=n_objectives, xl=-1, xu=1, type_var=float)

    def _evaluate(self, x: list[np.array], out, *args, **kwargs):
        #if POP_SIZE != len(x):
        #    print(f"WARNING: POP_SIZE != len(x) in evaluation step (this happens sometimes do not see why)\n"
        #          f"pop size: {POP_SIZE}; len x:{len(x)}")
        # Initialize
        dict_enemies = {}
        # Get fitness for each enemy
        for enemy in self.enemies:
            self.env.update_parameter('enemies', [enemy])

            dict_enemies[enemy] = []
            for individual_id in range(len(x)):
                dict_enemies[enemy].append(
                    simulation(self.env, x[individual_id], inverted_fitness=True, fitness_function=individual_gain))

        # Return fitness outputs for enemies
        objectives_fitness = {
            "objective_hard": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [1, 6]]) for ind_id in
                               range(len(x))],
            "objective_medium": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [2, 5, 8]]) for ind_id in
                                 range(len(x))],
            "objective_easy": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [3, 4, 7]]) for ind_id in
                               range(len(x))],
        }

        out["F"] = np.column_stack([objectives_fitness[key] for key in objectives_fitness.keys()])
        return out

In [5]:
env, n_genes = init_env('local_search_test', ENEMIES, n_hidden_neurons)
env.update_parameter('multiplemode', 'no')
problem = objectives(
        env=env,
        n_genes=n_genes,
        enemies=ENEMIES,
        n_objectives=3)

In [11]:
x_curr = solutions[[0],:]
x_best = np.copy(x_curr)

f_curr = problem._evaluate(x_curr, {})['F']
f_best, f_best_orig = np.copy(f_curr), np.copy(f_curr)

n_nbrs = 25

print(f_best)

[[0.00607702 0.00324837 0.00391076]]


In [7]:
# This is the function that needs to be defined, this is how you find neighbors of solution x
# For now, it gives a neighborhood where every gene has one 'mutation' or difference (the np.random.normal)
# I could not find clear examples of algorithms for the neighbor generating

def neighborhood(x, n):
    nbrhood = []
    for i in range(n):
        new_nbr = np.copy(x) * np.random.normal(size=x.shape[1])
        nbrhood.append(new_nbr)
    
    return nbrhood

In [12]:
done = False
solutionsChecked = 0

while not done:
    Neighborhood = neighborhood(x_curr, n_nbrs)
    for s in Neighborhood:
        solutionsChecked += 1
        
        try:
            eval_s = problem._evaluate(s, {})['F']
        except:
            # print("Infeasible solution evaluated")
            continue
        
        if eval_s[0].mean() > f_best[0].mean():
            x_best = np.copy(s)
            f_best = np.copy(eval_s)
    
    if np.array_equal(f_best, f_curr):
        done = True
    else:
        x_curr = np.copy(x_best)
        f_curr = np.copy(f_best)
        
        print(f"Total number of solutions checked: {solutionsChecked}")
        print(f"Best value found so far: {f_best}")
        
print(f"Final number of solutions checked: {solutionsChecked}")
print(f"New solution fitness: {f_best}")
print(f"Initial solution fitness: {f_best_orig}")

Total number of solutions checked: 25
Best value found so far: [[0.0069372  0.00695339 0.00689635]]
Total number of solutions checked: 50
Best value found so far: [[0.00692818 0.00694971 0.00691067]]
Final number of solutions checked: 75
New solution fitness: [[0.00692818 0.00694971 0.00691067]]
Initial solution fitness: [[0.00607702 0.00324837 0.00391076]]


In [ ]:
x_curr = solutions[[0],:]
x_best = np.copy(x_curr)

f_curr = problem._evaluate(solutions, {})['F']
f_best = np.copy(f_curr)

print(f_curr)

In [14]:
t = 100000
M = 10
k = 0
n_nbrs = 25

solutionsChecked = 0

done = False
while not done:
    if t < 1:
        done = True
    
    m = 0
    while m < M:
        solutionsChecked += 1
        # print(f"k = {k}, m = {m}, s = {solutionsChecked} \n")
        
        N = neighborhood(x_curr, n_nbrs)
        idx = np.random.randint(len(N))
        s = N[idx]        
        
        try:
            eval_s = problem._evaluate(s, {})['F']
        except Exception as e:
            break
        
        if eval_s[0].mean() >= f_best[0].mean():
            x_best = np.copy(s)
            f_best = np.copy(eval_s)
        else:
            p = np.exp(-(f_curr[0].mean() - eval_s[0].mean()) / t)
            test_p = np.random.uniform(0, 1)
            if test_p <= p:
                x_best = np.copy(s)
                f_best = np.copy(eval_s)
            else:
                print(test_p, p)
        
        m += 1
        
    k += 1
    t = 0.8*t
    print(f"Temperature lowered to {t} (s= {solutionsChecked})")
    
print(f"Final number of solutions checked: {solutionsChecked}")
print(f"New solution fitness: {f_best}")
print(f"Initial solution fitness: {f_best_orig}")

KeyboardInterrupt: 